# Chatbot using char embedding and an LSTM layer
- Doesn't require exact words defined in intents.json (char tokenizing vs word tokenizing)
- Tolerant to typos and words not found in intents.json
- Slower training and predicting times compared to using dense layers or word tokenization
- Better recognition of unknown phrases

# Setup

In [1]:
import numpy as np
import random
import json
import nltk
import tensorflow as tf
from tensorflow import keras
import pickle

# Read and parse intents.json

In [21]:
with open('intents.json') as file:
    data = json.load(file)

words = []
labels = []
docs_x = []
docs_y = []

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        wrds = [w.lower() for w in wrds if w != '?']
        words.extend(wrds)
        docs_x.append(' '.join(wrds))
        docs_y.append(intent['tag'])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])


words = sorted(list(set(words)))
tokenizer.fit_on_texts(' '.join(words))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]
training_empty = np.array([0. for _ in range(len(tokenizer.word_index)+1)])

for x, doc in enumerate(docs_x):
    
#     doc_onehot_sequence = tokenizer.texts_to_matrix(doc)
    doc_sequence = np.array(tokenizer.texts_to_sequences(doc))
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
#     training.append(doc_onehot_sequence)
    training.append(doc_sequence)
    output.append(output_row)
    
training_padded = keras.preprocessing.sequence.pad_sequences(training)
output = np.array(output)

# Define model and train (this time with embedding matrix and an LSTM layer)

In [3]:
model = keras.models.Sequential([
    
    keras.layers.Embedding(len(tokenizer.word_index)+1,8, mask_zero=True),
    keras.layers.LSTM(16),
    keras.layers.Dense(len(labels), activation='softmax')
    
])

In [5]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')
history = model.fit(x=training_padded,
                    y=output,
                    batch_size=8,
                    epochs=1000,
                    callbacks=[keras.callbacks.ReduceLROnPlateau(monitor='loss',factor=0.7,patience=30),
                               keras.callbacks.EarlyStopping(monitor='loss', patience=150, restore_best_weights=True)])

Epoch 1/500
7/7 [==============================] - 1s 82ms/step - loss: 0.0239 - accuracy: 0.9815
Epoch 2/500
7/7 [==============================] - 1s 80ms/step - loss: 0.0211 - accuracy: 0.9815
Epoch 3/500
7/7 [==============================] - 1s 82ms/step - loss: 0.0261 - accuracy: 0.9630
Epoch 4/500
7/7 [==============================] - 1s 80ms/step - loss: 0.0191 - accuracy: 0.9815
Epoch 5/500
7/7 [==============================] - 1s 81ms/step - loss: 0.0183 - accuracy: 0.9815
Epoch 6/500
7/7 [==============================] - 1s 80ms/step - loss: 0.0173 - accuracy: 0.9815
Epoch 7/500
7/7 [==============================] - 1s 80ms/step - loss: 0.0169 - accuracy: 0.9815
Epoch 8/500
7/7 [==============================] - 1s 79ms/step - loss: 0.0161 - accuracy: 0.9815
Epoch 9/500
7/7 [==============================] - 1s 79ms/step - loss: 0.0156 - accuracy: 0.9815
Epoch 10/500
7/7 [==============================] - 1s 79ms/step - loss: 0.0152 - accuracy: 0.9815
Epoch 11/500
7/7 [=

7/7 [==============================] - 1s 79ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 84/500
7/7 [==============================] - 1s 80ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 85/500
7/7 [==============================] - 1s 81ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 86/500
7/7 [==============================] - 1s 81ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 87/500
7/7 [==============================] - 1s 81ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 88/500
7/7 [==============================] - 1s 82ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 89/500
7/7 [==============================] - 1s 81ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 90/500
7/7 [==============================] - 1s 80ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 91/500
7/7 [==============================] - 1s 79ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 92/500
7/7 [==============================] - 1s 79ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 93/500
7/7 [=====

7/7 [==============================] - 1s 84ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 166/500
7/7 [==============================] - 1s 78ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 167/500
7/7 [==============================] - 1s 79ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 168/500
7/7 [==============================] - 1s 79ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 169/500
7/7 [==============================] - 1s 80ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 170/500
7/7 [==============================] - 1s 79ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 171/500
7/7 [==============================] - 1s 82ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 172/500
7/7 [==============================] - 1s 81ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 173/500
7/7 [==============================] - 1s 83ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 174/500
7/7 [==============================] - 1s 79ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 175/500


7/7 [==============================] - 1s 76ms/step - loss: 6.8525e-04 - accuracy: 1.0000
Epoch 245/500
7/7 [==============================] - 1s 78ms/step - loss: 6.8113e-04 - accuracy: 1.0000
Epoch 246/500
7/7 [==============================] - 1s 78ms/step - loss: 6.7584e-04 - accuracy: 1.0000
Epoch 247/500
7/7 [==============================] - 1s 80ms/step - loss: 6.7086e-04 - accuracy: 1.0000
Epoch 248/500
7/7 [==============================] - 1s 78ms/step - loss: 6.6616e-04 - accuracy: 1.0000
Epoch 249/500
7/7 [==============================] - 1s 77ms/step - loss: 6.6116e-04 - accuracy: 1.0000
Epoch 250/500
7/7 [==============================] - 1s 75ms/step - loss: 6.5646e-04 - accuracy: 1.0000
Epoch 251/500
7/7 [==============================] - 1s 78ms/step - loss: 6.5242e-04 - accuracy: 1.0000
Epoch 252/500
7/7 [==============================] - 1s 77ms/step - loss: 6.4630e-04 - accuracy: 1.0000
Epoch 253/500
7/7 [==============================] - 1s 80ms/step - loss: 6.42

7/7 [==============================] - 1s 76ms/step - loss: 4.1438e-04 - accuracy: 1.0000
Epoch 323/500
7/7 [==============================] - 1s 77ms/step - loss: 4.1274e-04 - accuracy: 1.0000
Epoch 324/500
7/7 [==============================] - 1s 77ms/step - loss: 4.1112e-04 - accuracy: 1.0000
Epoch 325/500
7/7 [==============================] - 1s 75ms/step - loss: 4.0898e-04 - accuracy: 1.0000
Epoch 326/500
7/7 [==============================] - 1s 77ms/step - loss: 4.0749e-04 - accuracy: 1.0000
Epoch 327/500
7/7 [==============================] - 1s 76ms/step - loss: 4.0557e-04 - accuracy: 1.0000
Epoch 328/500
7/7 [==============================] - 1s 77ms/step - loss: 4.0317e-04 - accuracy: 1.0000
Epoch 329/500
7/7 [==============================] - 1s 76ms/step - loss: 4.0196e-04 - accuracy: 1.0000
Epoch 330/500
7/7 [==============================] - 1s 76ms/step - loss: 3.9994e-04 - accuracy: 1.0000
Epoch 331/500
7/7 [==============================] - 1s 77ms/step - loss: 3.98

7/7 [==============================] - 1s 76ms/step - loss: 3.2604e-04 - accuracy: 1.0000
Epoch 401/500
7/7 [==============================] - 1s 76ms/step - loss: 3.2557e-04 - accuracy: 1.0000
Epoch 402/500
7/7 [==============================] - 1s 79ms/step - loss: 3.2458e-04 - accuracy: 1.0000
Epoch 403/500
7/7 [==============================] - 1s 77ms/step - loss: 3.2389e-04 - accuracy: 1.0000
Epoch 404/500
7/7 [==============================] - 1s 77ms/step - loss: 3.2290e-04 - accuracy: 1.0000
Epoch 405/500
7/7 [==============================] - 1s 76ms/step - loss: 3.2237e-04 - accuracy: 1.0000
Epoch 406/500
7/7 [==============================] - 1s 77ms/step - loss: 3.2149e-04 - accuracy: 1.0000
Epoch 407/500
7/7 [==============================] - 1s 76ms/step - loss: 3.2075e-04 - accuracy: 1.0000
Epoch 408/500
7/7 [==============================] - 1s 79ms/step - loss: 3.1992e-04 - accuracy: 1.0000
Epoch 409/500
7/7 [==============================] - 1s 78ms/step - loss: 3.19

7/7 [==============================] - 1s 80ms/step - loss: 2.8884e-04 - accuracy: 1.0000
Epoch 479/500
7/7 [==============================] - 1s 75ms/step - loss: 2.8857e-04 - accuracy: 1.0000
Epoch 480/500
7/7 [==============================] - 1s 78ms/step - loss: 2.8828e-04 - accuracy: 1.0000
Epoch 481/500
7/7 [==============================] - 1s 75ms/step - loss: 2.8802e-04 - accuracy: 1.0000
Epoch 482/500
7/7 [==============================] - 1s 76ms/step - loss: 2.8774e-04 - accuracy: 1.0000
Epoch 483/500
7/7 [==============================] - 1s 76ms/step - loss: 2.8747e-04 - accuracy: 1.0000
Epoch 484/500
7/7 [==============================] - 1s 75ms/step - loss: 2.8724e-04 - accuracy: 1.0000
Epoch 485/500
7/7 [==============================] - 1s 75ms/step - loss: 2.8689e-04 - accuracy: 1.0000
Epoch 486/500
7/7 [==============================] - 1s 78ms/step - loss: 2.8664e-04 - accuracy: 1.0000
Epoch 487/500
7/7 [==============================] - 1s 78ms/step - loss: 2.86

# Setup predictions

In [6]:
def str_to_sequence(inp):
    wrds = nltk.word_tokenize(inp)
    wrds = [w.lower() for w in wrds if w != '?']
    inp_text = ' '.join(wrds)
    inp_seq = tokenizer.texts_to_sequences(inp_text)
    return np.array(inp_seq).reshape(1,-1,1)

def model_response(pred, labels, data):
    if max(pred)<0.9:
        uncertain_responses = ['I dont understand', 'Sorry I dont get what you\'re saying', 'Could you word that differently?']
        print(f"Chatbot: {random.choice(uncertain_responses)}")
    else:
        pred_idx = np.argmax(pred)
        label = labels[pred_idx]

        for intent in data['intents']:
            if label == intent['tag']:
                print(f"Chatbot: {random.choice(intent['responses'])}")
                break

# Define chat loop and chat!

In [12]:
def chat():
    print('Start talking with the bot! (type quit to stop)\n')
    while True:
        inp = input("You: ")
        if inp.lower()=='quit':
            break
        inp = ''.join([c for c in inp if c in tokenizer.word_index.keys()])
        inp_seq = str_to_sequence(inp)
        pred = model.predict(inp_seq).reshape(-1)
        model_response(pred,labels,data)

In [23]:
chat()

Start talking with the bot! (type quit to stop)

You: hello!
Chatbot: Good to see you again!
You: how are you?
Chatbot: Good to see you again!
You: what do you sell?
Chatbot: We sell chocolate chip cookies for $2!
You: do you have a name?
Chatbot: I'm Chatbot!
You: how old are you?
Chatbot: I am 18 years old!
You: do you deliver?
Chatbot: Sure, call this number and make your delivery!
You: hellwo!
Chatbot: Hi there, how can I help?
You: do u make reservation
Chatbot: I dont understand
You: do u accept reservation
Chatbot: We'd gladly take your reservation, call this number
You: wat is ur name
Chatbot: I'm Chatbot!
You: goodby chatbot!
Chatbot: Goodbye!
You: quit
